Convert checkpointed model from GCP VM to Keras saved model for deployment in
irrigation_detection_inference.ipynb

In [ ]:
from tensorflow.python import saved_model
def convert_tf_model(input_model_folder, output_model_folder):

    import tensorflow as tf
    from tensorflow.keras.layers import Dense, Dropout, LSTM
    from tensorflow.keras import Model, Sequential
    from tensorflow.keras import layers
    from zipfile import ZipFile

    # Define model encoder
    def _transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
        # Normalization and Attention
        x = layers.LayerNormalization(epsilon=1e-6)(inputs)
        x = layers.MultiHeadAttention(
            key_dim=head_size, num_heads=num_heads, dropout=dropout
        )(x, x)
        x = layers.Dropout(dropout)(x)
        res = x + inputs

        # Feed Forward Part
        x = layers.LayerNormalization(epsilon=1e-6)(res)
        x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
        x = layers.Dropout(dropout)(x)
        x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
        return x + res

    # Build model
    def _build_model(
        input_shape,
        head_size,
        num_heads,
        ff_dim,
        num_transformer_blocks,
        mlp_units,
        dropout=0,
        mlp_dropout=0,
    ):
        inputs = tf.keras.Input(shape=input_shape)
        x = inputs
        for _ in range(num_transformer_blocks):
            x = _transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

        x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
        for dim in mlp_units:
            x = layers.Dense(dim, activation="relu")(x)
            x = layers.Dropout(mlp_dropout)(x)

        n_classes = 2
        outputs = layers.Dense(n_classes, activation='softmax')(x)

        return Model(inputs, outputs)

    # Return model
    def _return_model():

        input_shape = (36,1)

        model = _build_model(
                        input_shape,
                        head_size=64,
                        num_heads=4,
                        ff_dim=4,
                        num_transformer_blocks=4,
                        mlp_units=[32],
                        mlp_dropout=0.4,
                        dropout=0.25,
                        )

        lr = 1e-4
        optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=lr)


        with ZipFile(input_model_folder, 'r') as zip:
            zip.extractall()
            print('Done')

        prev_checkpoint_prefix = input_model_folder.replace('.zip', '')
        checkpoint = tf.train.Checkpoint(
            optimizer=optimizer,
            model=model,
        )
        print(f'Loading existing checkpoint: {tf.train.latest_checkpoint(prev_checkpoint_prefix)}')
        status = checkpoint.restore(tf.train.latest_checkpoint(prev_checkpoint_prefix)).expect_partial()
        # print(status.assert_existing_objects_matched())

        model.compile(optimizer)

        return model

    # Save model locally
    def _save_model(model):
        model.save(output_model_folder)

    # Upload converted model to GCP
    def _upload_files():

        from google.cloud import storage
        from google.colab import auth
        from glob import glob
        import os
        # Authenticate Google account
        auth.authenticate_user()

        bucket_name = 'gee_irrigation_detection'
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)

        """Upload files to GCP bucket."""
        files = glob(output_model_folder + "/**", recursive=True)
        for file in files:
            if os.path.isfile(file):
                print(file)
                blob = bucket.blob(f'saved_models/{file}')
                blob.upload_from_filename(file)

        return f'Uploaded to "{bucket_name}" bucket.'



    # Execute function
    model = _return_model()
    _save_model(model)
    _upload_files()



# Upload folder containing Transformer model checkpoint, data, and index files
# Folder will be a zip file with a datetime as a name
input_model_folder = '20221226-184032.zip'

# Define out folder
output_model_folder = 'transformer_trained_2020'

convert_tf_model(input_model_folder, output_model_folder)

Done
Loading existing checkpoint: 20221226-184032/epoch_19_top_min_f1_0.9150-8


transformer_trained_2020/variables/variables.index
transformer_trained_2020/variables/variables.data-00000-of-00001
transformer_trained_2020/saved_model.pb
transformer_trained_2020/keras_metadata.pb


In [ ]:
def compare_annual_models():

    from google.cloud import storage
    from google.colab import auth
    import numpy as np
    from tqdm import tqdm

    !pip install rasterio
    import rasterio

    # Authenticate Google account
    auth.authenticate_user()

    model_0 = 'transformer_trained_2020'
    model_1 = 'transformer_trained_2021'

    # Initialize client and find folders in bucket
    storage_client = storage.Client()
    bucket_name = 'gee_irrigation_detection'
    blobs = storage_client.list_blobs(bucket_name, prefix=f'model_predictions')

    # Find all images
    pred_images = [blob.name for blob in blobs if '.tif' in blob.name]

    # Find predictions by model
    model_1_predictions = sorted([i for i in pred_images if model_0 in i])
    model_2_predictions = sorted([i for i in pred_images if model_1 in i])

    misaligned_count = 0
    total_count = 0

    # Compare predictions across models
    for ix, image in enumerate(tqdm(model_1_predictions)):

        img_0 = rasterio.open(f'gs://{bucket_name}/{image}').read()
        img_1 = rasterio.open(f'gs://{bucket_name}/{model_2_predictions[ix]}').read()

        print(img_0 - img_1)

        misaligned_count += np.count_nonzero(img_0 - img_1)
        total_count += img_0.shape[1] * img_0.shape[2]

    print(f'Total misaligned: {np.round(misaligned_count/total_count, decimals=3)} ({misaligned_count}/{total_count})')

compare_annual_models()